# Bermudan Option Pricing via a Pure Dual Approach
Implementation based on the paper "A pure dual approach for hedging Bermudan options" by Alfonsi, Kebaier, and Lelong.

In [36]:
import numpy as np
from sklearn.linear_model import LinearRegression

def simulate_gbm(S0, r, sigma, T, N, M):
    dt = T / N
    paths = np.zeros((M, N+1))
    paths[:, 0] = S0
    for t in range(1, N+1):
        paths[:, t] = paths[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * np.random.randn(M))
    return paths

def compute_martingale_approximation(S0, K, r, sigma, T, N, M, basis_degree=4):
    # Simulate paths
    paths = simulate_gbm(S0, r, sigma, T, N, M)
    Z = np.maximum(K - paths, 0)  # Put option payoff
    
    delta_M = np.zeros((M, N+1))
    
    for n in range(N-1, -1, -1):
        #print(f"Processing time step n = {n}")
        
        # Compute max target values for each path
        max_values = np.zeros(M)
        if n+1 <= N:
            current_max = Z[:, n+1].copy()  # j = n+1 case
            for j in range(n+2, N+1):
                # Sum delta_M from n+2 to j
                sum_delta = np.sum(delta_M[:, n+2:j+1], axis=1)
                current_val = Z[:, j] - sum_delta
                current_max = np.maximum(current_max, current_val)
            max_values = current_max
        else : print("this never happens")
        
        # Prepare basis functions for X_{n+1} and adjust them
        X_next = paths[:, n+1]
        # Create polynomial basis up to basis_degree
        basis_next = np.column_stack([X_next**k for k in range(1, basis_degree+1)])
        
        X_current = paths[:, n]
        # Basis for X_n includes intercept
        basis_current = np.column_stack([X_current**k for k in range(0, basis_degree+1)])
        
        # Compute adjusted basis by subtracting conditional expectations
        adjusted_basis = np.zeros_like(basis_next)
        for k in range(basis_next.shape[1]):
            model = LinearRegression()
            model.fit(basis_current, basis_next[:, k])
            pred = model.predict(basis_current)
            adjusted_basis[:, k] = basis_next[:, k] - pred
        
        # Regress (max_values - Z_n) on adjusted_basis
        residuals = max_values - Z[:, n]
        model = LinearRegression(fit_intercept=False)
        model.fit(adjusted_basis, residuals)
        delta_M[:, n+1] = model.predict(adjusted_basis)
    
    # Compute the martingale as cumulative sums of delta_M
    M_martingale = np.cumsum(delta_M, axis=1)
    return M_martingale, delta_M, paths, Z


In [28]:
def compute_lower_bound(paths, K, r, T, N, basis_degree=4):
    dt = T / N
    discount = np.exp(-r * dt)
    M = paths.shape[0]
    cashflows = np.maximum(K - paths[:, -1], 0)  # Put option payoff at maturity

    for t in range(N-1, 0, -1):
        itm = np.where(paths[:, t] < K)[0]  # In-the-money paths
        X = paths[itm, t]
        Y = cashflows[itm] * discount

        # Regression on polynomial basis
        basis = np.column_stack([X**d for d in range(basis_degree + 1)])
        model = LinearRegression()
        model.fit(basis, Y)
        continuation = model.predict(basis)

        exercise = K - X > continuation
        cashflows[itm[exercise]] = K - X[exercise]
        cashflows *= discount  # Discount all cashflows one step back

    return np.mean(cashflows)

In [33]:
# Example parameters
M, T, n = 100_000, 1.0, 100
r, vol, S0 = 0.06, 0.2, 36.0
K = 40
dt   = T / n

# Generate paths, martingale, and payoffs
M_martingale, delta_M, paths, Z = compute_martingale_approximation(
    S0, K, r, vol, T, n, M
)

Processing time step n = 99
Processing time step n = 98
Processing time step n = 97
Processing time step n = 96
Processing time step n = 95
Processing time step n = 94
Processing time step n = 93
Processing time step n = 92
Processing time step n = 91
Processing time step n = 90
Processing time step n = 89
Processing time step n = 88
Processing time step n = 87
Processing time step n = 86
Processing time step n = 85
Processing time step n = 84
Processing time step n = 83
Processing time step n = 82
Processing time step n = 81
Processing time step n = 80
Processing time step n = 79
Processing time step n = 78
Processing time step n = 77
Processing time step n = 76
Processing time step n = 75
Processing time step n = 74
Processing time step n = 73
Processing time step n = 72
Processing time step n = 71
Processing time step n = 70
Processing time step n = 69
Processing time step n = 68
Processing time step n = 67
Processing time step n = 66
Processing time step n = 65
Processing time step

In [38]:
for k in range(35,46):# The true adjusted payoff at time 0 is the max over Z_j - sum of delta_M from 1 to j
    M_martingale, delta_M, paths, Z = compute_martingale_approximation(
        S0, k, r, vol, T, n, M
    )
    upper_bounds = np.zeros(M)
    for j in range(1, n+1):
        adjusted = Z[:, j] - np.sum(delta_M[:, 1:j+1], axis=1)
        upper_bounds = np.maximum(upper_bounds, adjusted)

    upper_bound_estimate = np.mean(upper_bounds)
    lower_bound_estimate = compute_lower_bound(paths, k, r, T, n, basis_degree=4)
    U_0 = np.max(Z - M_martingale, axis=1)
    bermudan_price = np.exp(-r * T) * np.mean(U_0)

    print(f"for K = {k} LB : {lower_bound_estimate}, UB : {upper_bound_estimate}]")

for K = 35 LB : 1.6462860984206484, UB : 1.940695431001385]
for K = 36 LB : 2.0840095157436522, UB : 2.4082246768293363]
for K = 37 LB : 2.5877523536760125, UB : 2.9317078356572304]
for K = 38 LB : 3.1458289751667308, UB : 3.49409666273451]
for K = 39 LB : 3.7730692840648605, UB : 4.108322096954641]
for K = 40 LB : 4.494042141735317, UB : 4.7930766952997566]
for K = 41 LB : 5.259614356543553, UB : 5.536509487903007]
for K = 42 LB : 6.106780355314029, UB : 6.3337326232473945]
for K = 43 LB : 7.011011908894566, UB : 7.160429743262684]
for K = 44 LB : 7.985117125521584, UB : 8.08013783629658]
for K = 45 LB : 8.971237359265217, UB : 9.020648069374971]


Bermudan Put Option Price: 8.5063
